# 20170802 Test Puck C Hene

Results here: 
  
- EXPERIMENT #: 10
- Input fiber: 200 micron
- Filter: Hene
- Location: 5th floor fiber lab
- Iris: Yes
- Stage: Yes
- Camera: Yes
- Date: August 3rd
- Stage: Yes, automated
- Fully automated

In [39]:
from __future__ import print_function
import pandas as pd
import numpy as np
import datetime
import matplotlib.dates as mdates

# ---------- Graphics ------------
# matplotlib
import seaborn as sns; sns.set()
sns.set_context("poster",font_scale=1.2,rc={"font":"helvetica"});
sns.set_style("white"); #sns.set_style("ticks")
cp = sns.color_palette("colorblind") #sns.palplot(current_palette)

%matplotlib inline
#%matplotlib osx
#%matplotlib nbagg
%config InlineBackend.figure_format = "retina"
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
#rcParams["text.usetex"] = True #uncomment to use tex. Slow, but pretty
#rcParams["font.weight"] = 900

rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family'] = 'STIXGeneral'
rcParams['font.weight'] = "normal"
rcParams["axes.formatter.useoffset"] = False
rcParams['xtick.major.width']=1
rcParams['xtick.major.size']=4
rcParams['xtick.minor.width']=0.5
rcParams['xtick.minor.size']=2
rcParams['xtick.direction'] = "in"
rcParams['ytick.direction'] = "in"
rcParams['ytick.major.width']=1
rcParams['ytick.major.size']=4
rcParams['ytick.minor.width']=0.5
rcParams['ytick.minor.size']=2

#%matplotlib nbagg

# Bokeh
from bokeh.io import output_notebook
output_notebook()

# ---------- Extras --------------
#import os
#import glob
#import astropy
from astropy import constants as aconst
#import exodata
#import quantities as pq
#exocat = exodata.load_db_from_url()
#exocat.searchPlanet('koi-13b')

# ————— Fitting -------------------
from lmfit import minimize, Parameters, fit_report
import lmfit
#from statsmodels.tsa.base.datetools import dates_from_str
#quarterly = dates_from_str(quarterly)

# ----------- Personal libraries -------
from fitsimagefli import FitsImageFLI
import gkfit
import gklib as gk
import gkastro as gkastro
from gkplt import gkplt21
from bokehlinked import BokehLinkedPanel, BokehLinkedPanelCS

# ----------- HPF ----------------------
import tel_plot_help as picx; reload(picx)
import hpf_static; reload(hpf_static)
from hpfteldata import HPFTelData

Loading BokehJS ...

In [27]:
import os
from astropy.modeling import models, fitting
import butter
import gkfit
reload(gkfit)
import fitsimg
reload(fitsimg)
import photutils
import glob
reload(gkastro)
import fitsimg
reload(fitsimg)
import phothelp
reload(phothelp)
from astropy.modeling import models, fitting

# Need to find the edge of the circle, not use a factor

class FRDImg(object):
    """
    A class to analyze FRDImages
    """
    
    def __init__(self,filename):
        self.filename   = filename
        self.basename   = self.filename.split(os.sep)[-1]
        self.fimg       = fitsimg.FitsImageFLI(self.filename)
        
    def run(self,
            plot_suffix,
            plot_folder,
            get_rad_at_EE = 0.90,
            MAXRAD_FACTOR = 0.7,
            fwzm_z=20):
        """
        Run the analysis
        
        INPUT:
            plot_suffix = "f01"
            plot_folder = "results/"
            get_rad_at_EE = 0.90
            FACTOR = 0.7
            fwzm_z=20

        OUTPUT:
            Saves files to folders  
        """
        gkastro.make_dir(plot_folder)
        fig, ax = plt.subplots(ncols=3,nrows=2,figsize=(20,15))
        
        ax_img         = ax.flat[0]
        ax_img_bkg     = ax.flat[1]
        ax_img_bkg_sub = ax.flat[2]
        ax_cut         = ax.flat[3]
        ax_ee          = ax.flat[4]
        ax_img_bkg_sub2= ax.flat[5]

        # -------------
        # Plot #1: Normal image
        x,y = self.fimg.get_centroid(plot_lines=True,ax=ax_img)
        self.fimg.ax.set_title("Bias + Dark Subtracted (Hist stretch)")
        #cbar_ax = fig.add_axes([0.38, 0.59, 0.015, 0.3],label="Counts")
        #fig.colorbar(self.fimg.im, cax=cbar_ax)
        
        # -------------
        # Plot #2: Background estimation
        x,y = self.fimg.get_centroid()
        print(x,y)
        CBS = phothelp.CircularBackgroundSubractor(self.fimg.data,x=x,y=y,r=400)
        data_back_sub = CBS.subtract_background(plot_background=True,ax=ax_img_bkg)
        self.subtracted_value = CBS.subtracted_value
        
        # -------------
        # Plot #3: Background subtracted image
        self.fimg.data = data_back_sub
        self.fimg.plot(ax=ax_img_bkg_sub,colorbar=False,title="Bias + Dark + Background subtracted (linear stretch)",stretch="linear")
        # add colorbar
        #cbar_ax = fig.add_axes([0.38, 0.09, 0.015, 0.3],label="Counts")
        #fig.colorbar(self.fimg.im, cax=cbar_ax)
        # Get centroid again after taking out the background
        x,y = self.fimg.get_centroid()        
        
        
        # -------------
        # Plot #4: Cuts
        cut_x, self.fwhm_x = self.fimg.get_centroid_line_cut(line="X",return_FWHM=True,plot=True,ax=ax_cut,use_butter_filter=True,butter_cutoff_freq=0.03,fwzm_z=fwzm_z)
        cut_y, self.fwhm_y = self.fimg.get_centroid_line_cut(line="Y",return_FWHM=True,plot=True,ax=ax_cut,use_butter_filter=True,butter_cutoff_freq=0.03,fwzm_z=fwzm_z)
        
        ax_cut.plot(butter.low_pass_butter(cut_x,cutoff_freq=0.03))
        ax_cut.plot(butter.low_pass_butter(cut_y,cutoff_freq=0.03))
        ax_cut.set_ylim(0,64000.)

        # -------------
        # Plot #4: Encircled energy plot
        
        self.max_radii_for_EE = ((self.fwhm_x + self.fwhm_y)/2.)*MAXRAD_FACTOR
        radii = np.arange(1,self.max_radii_for_EE)
        df, self.r_ee = phothelp.get_encircled_energy_and_rad_at_EE(self.fimg.data,
                                                               self.fimg.xcenter,
                                                               self.fimg.ycenter,
                                                               radii,
                                                               plot=True,
                                                               ax=ax_ee,
                                                               get_rad_at_EE=get_rad_at_EE)
        ax_ee.set_xlim(0,500)
        ax_ee.set_xlabel("Radii (pixels)")
        
        # Overplot aperture in figure $ with EE
        aper = photutils.CircularAperture((x,y),r=self.r_ee)
        aper.plot(ax=ax_img_bkg_sub,color="green")
        ax_img_bkg_sub.annotate("EE"+str(100*get_rad_at_EE)+"%",xy=(x,y+self.r_ee),color="white",fontsize=8)
        # overplot max aperture 
        aper = photutils.CircularAperture((x,y),r=self.max_radii_for_EE)
        aper.plot(ax=ax_img_bkg_sub,color="red")
        ax_img_bkg_sub.annotate("EE100% @ r = "+str(MAXRAD_FACTOR*2)+" HWHM",xy=(x,y+self.max_radii_for_EE),color="white",fontsize=8)
        
        # Hist plot
        self.fimg.plot(ax=ax_img_bkg_sub2,colorbar=False,title="Bias + Dark + Background subtracted (Hist stretch)",stretch="hist")
        aper = photutils.CircularAperture((x,y),r=self.r_ee)
        aper.plot(ax=ax_img_bkg_sub2,color="green")
        ax_img_bkg_sub2.annotate("EE"+str(100*get_rad_at_EE)+"%",xy=(x,y+self.r_ee),color="white",fontsize=8)
        # overplot max aperture 
        aper = photutils.CircularAperture((x,y),r=self.max_radii_for_EE)
        aper.plot(ax=ax_img_bkg_sub2,color="red")
        ax_img_bkg_sub2.annotate("EE100% @ r = "+str(MAXRAD_FACTOR*2)+" HWHM",xy=(x,y+self.max_radii_for_EE),color="white",fontsize=8)

        fig.tight_layout()
        fig.subplots_adjust(top=0.9)
        fig.suptitle("FRD Overview Plot, file: "+self.basename,fontsize=20)
        fig.savefig(plot_folder + self.basename+"_"+plot_suffix+".png")
        print("Saved file:",plot_folder + self.basename+"_"+plot_suffix+".png")
        df.to_csv(plot_folder + self.basename+"_"+plot_suffix+".csv")
        print("Saved file:",plot_folder + self.basename+"_"+plot_suffix+".csv")
        
        
class AnalyzeFRDImages(object):
    """
    Analyze a set of FRDImages.
    
    NOTES:
    See *analyze_all_frames()*
    """
    # CONSTANTS
    PIXEL_SCALE = 0.013 # mm
    F_LENGTH_IN = 20.0  # mm
    
    def __init__(self,
                 fitsfiles,
                 plot_suffix,
                 plot_folder,
                 setup_csv_file,
                 FOLDER_CSV_SAVE,
                 motorized=True,
                 get_rad_at_EE=0.96,
                 MAXRAD_FACTOR=0.54):
        self.fitsfiles        = fitsfiles
        self.plot_suffix      = plot_suffix
        self.plot_folder      = plot_folder
        self.setup_csv_file   = setup_csv_file
        self.FOLDER_CSV_SAVE  = FOLDER_CSV_SAVE
        self.MOTORIZED        = True # using motorized iris or not ?
        self.f_number_names   = pd.Series([filepath.FilePath(filename).basename.split("_")[2] for filename in self.fitsfiles]).unique()
        self.NUM_F_NUMBERS    = len(self.f_number_names) # Number of different F-numbers measured
        self.get_rad_at_EE    = get_rad_at_EE # What EE to get
        self.MAXRAD_FACTOR    = MAXRAD_FACTOR # The factor to multiply the average of FWHM_X and FWHM_Y to set as the maximum radius
        self.BACKUP_FILE_NAME = self.FOLDER_CSV_SAVE + self.plot_suffix + "_radii_fwhm_subtracted.npz"
        
    def _get_df(self,df_config,frat,startnum=0):
        """
        Get data frames, and finds all of the records for a given F input, and 
        calculates the input and output F-ratios.
        
        INPUT: 
            df_config - dataframe with the measurements
            frat      - fiber input measurement set
            
        OUTPUT:
        
        NOTES:
            When using the motorized setup, then need to use the line model to convert to mm.
            
        """
        # Filenames are f_psm_01_d_01.fits
        # Select only measurements for a given input F-number
        df = df_config[df_config.filename.isin(["f_psm_"+frat+"_d_"+str(i).zfill(2)+".fits" for i in range(1,12)])]
        df = df.reset_index(drop=True)
        df = df[startnum:]
        
        #if method=="FIRST":
        df["x_out_fiber_dist_delta"] = df["x_out_fiber_dist"].values[0] - df["x_out_fiber_dist"].values
        df["y_out_fiber_dist_delta"] = df["y_out_fiber_dist"].values - df["y_out_fiber_dist"].values[0]
        df["f_ratio_out"] = df["x_out_fiber_dist_delta"].values/(2.*df["y_out_fiber_dist_delta"].values)
        #elif method=="SEQUENTIAL"
        
        if self.MOTORIZED==True:
            # motorized setup: convert from percent to mm
            df["iris"] = self._percent2mm(df["iris_diam_perc"])
        df["f_ratio_in"]  = self.F_LENGTH_IN/df["iris"]
        return df
    
    def load_backup():
        backup_file = np.load(self.BACKUP_FILE_NAME)
        a, b, c = backup_file.items()
        radii_at_ee, fwhm, subtracted_values = a[1], b[1], c[1]
        return radii_at_ee, fwhm, subtracted_values

    def _calc_f_out(self,radii_at_EE,subtracted_values,save=True):
        """
        Calculate the F output F number for a given EE.
        
        INPUT:
            radii_at_EE - the radius (in pix) used to calculate the 
        
        setup_csv_file = "/Users/gks/Dropbox/mypylib/notebooks/HPF_FRD/frd_files_final_run_f01.csv"
        """
        filepath = gkastro.FilePath(self.setup_csv_file)
        filepath.add_prefix("df_config_")

        df_config = pd.read_csv(self.setup_csv_file)[0:len(self.fitsfiles)]
        df_config["radii_at_EE"] = radii_at_EE
        df_config["sub_values"]  = subtracted_values
        df_config["y_out_fiber_dist"] = df_config["radii_at_EE"]*self.PIXEL_SCALE
        df_config = pd.concat([self._get_df(df_config,frat=str(i).zfill(2)) for i in range(1,self.NUM_F_NUMBERS+1)],ignore_index=True)
        if save:
            df_config.to_csv(self.FOLDER_CSV_SAVE+filepath.basename)
            print("Saved to:",self.FOLDER_CSV_SAVE+filepath.basename)
        return df_config
    
    def _percent2mm(self,percent):
        """
        Converts between iris size in percent to mm
        """
        m_line = models.Linear1D(slope=0.333201810375,intercept=2.02718927701)
        print("LINE",m_line(percent))
        return m_line(percent)

    def analyze_all_frames(self):
        """
        Analyze all the frames, and saves as a .csv file

        OUTPUT:
        """
        plt.switch_backend("agg")
        self.radii_at_EE       = np.zeros(len(self.fitsfiles))
        self.fwhm              = np.zeros(len(self.fitsfiles))
        self.subtracted_values = np.zeros(len(self.fitsfiles))
        for i,fitsfile in enumerate(self.fitsfiles):
            frd = FRDImg(fitsfile)
            print("Analyzing file #",i,fitsfile)
            frd.run(get_rad_at_EE=self.get_rad_at_EE,
                    MAXRAD_FACTOR=self.MAXRAD_FACTOR,
                    plot_suffix=self.plot_suffix,
                    plot_folder=self.plot_folder);
            print("Radii at EE:",frd.r_ee)
            self.radii_at_EE[i] = frd.r_ee
            self.fwhm[i] = frd.max_radii_for_EE
            self.subtracted_values[i] = frd.subtracted_value
            np.savez(self.BACKUP_FILE_NAME,self.radii_at_EE,self.fwhm,self.subtracted_values)
            print("Saved backup to",self.BACKUP_FILE_NAME)
            print("FWHM",frd.max_radii_for_EE)
            print("")
        self.df_config = self._calc_f_out(self.radii_at_EE,self.subtracted_values,save=True)
        return self.df_config
    
    def _get_EE_in_input_cone(self,df_config,suffix):
        """
        Get encircled energy in the given input cone.
        """
        df_config["y_for_EE_in_input_cone"] = np.zeros(len(df_config))
        df_config["EE_in_input_cone"] = np.zeros(len(df_config))

        for i, filename in enumerate(df_config["filename"].values):
            print(i)
            print(filename)
            if (np.isfinite(df_config["f_ratio_out"].values[i])):
                filepath = gkastro.FilePath(filename)
                filepath.basename 
                df_ee = pd.read_csv(self.plot_folder + filepath.barename + "_out.fits_" + suffix + ".csv")

                df_config["y_for_EE_in_input_cone"].ix[i] = df_config["y_in_input"].values[i]/self.PIXEL_SCALE
                df_config["EE_in_input_cone"].ix[i]       = get_EE_from_rad_in_pix(df_ee,df_config["y_for_EE_in_input_cone"].ix[i])
            else:
                df_config["y_for_EE_in_input_cone"].ix[i] = np.nan
                df_config["EE_in_input_cone"].ix[i]       = np.nan
                print("Skip!")
        return df_config

#############################################################
    
def get_EE_from_rad_in_pix(df,radius_in_pix):
    return df[df["radii"] > radius_in_pix]["EE"].values[0]

def add_y_in_input(df_config):
    df_config["y_in_input"] = (df_config["f_ratio_out"]/df_config["f_ratio_in"])*df_config["y_out_fiber_dist"]
    return df_config

def resample_df_mean(df):
    chunks = gkastro.chunker(df,4)
    dff = pd.DataFrame()
    for chunk in chunks:
        dff = pd.concat([dff,pd.DataFrame(chunk[["f_ratio_in","f_ratio_out","EE_in_input_cone"]].median(skipna=True)).T],ignore_index=True)
    return dff

In [28]:
import glob
import filepath

# Folders
BASEFOLDER       = "/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/"
FOLDER_CSV_SETUP = BASEFOLDER+"ANALYSIS/CSV_SETUP/"
FOLDER_CSV_SAVE  = BASEFOLDER+"ANALYSIS/CSV_RESULTS/"
PLOT_FOLDER      = BASEFOLDER+"ANALYSIS/PLOTS/"

# Files
setup_csv_files  = glob.glob(FOLDER_CSV_SETUP+"*.csv")
fitsfiles_f01    = glob.glob(BASEFOLDER+"f01/pipelineout/*.fits")
fitsfiles_f02    = glob.glob(BASEFOLDER+"f02/pipelineout/*.fits")
#fitsfiles_f03    = glob.glob(BASEFOLDER+"f03/pipelineout/*.fits")

In [17]:
setup_csv_files

['/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/ANALYSIS/CSV_SETUP/setup_f01.csv',
 '/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/ANALYSIS/CSV_SETUP/setup_f02.csv']

In [18]:
plt.switch_backend("agg")
fig, ax= plt.subplots()

In [19]:
fitsfiles_f01[0:]

['/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f01/pipelineout/f_psm_01_d_01_out.fits',
 '/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f01/pipelineout/f_psm_01_d_02_out.fits',
 '/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f01/pipelineout/f_psm_01_d_03_out.fits',
 '/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f01/pipelineout/f_psm_01_d_04_out.fits',
 '/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f01/pipelineout/f_psm_02_d_01_out.fits',
 '/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f01/pipelineout/f_psm_02_d_02_out.fits',
 '/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f01/pipelineout/f_psm_02_d_03_out.fits',
 '/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f

# F1

In [5]:
AFRDImg.NUM_F_NUMBERS

NameError: name 'AFRDImg' is not defined

In [20]:
AFRDImg = AnalyzeFRDImages(fitsfiles=fitsfiles_f01,
                           plot_suffix="f01",
                           plot_folder=PLOT_FOLDER,
                           setup_csv_file=setup_csv_files[0],
                           FOLDER_CSV_SAVE=FOLDER_CSV_SAVE,
                           motorized=True,
                           MAXRAD_FACTOR=0.56)
AFRDImg.analyze_all_frames()

Analyzing file # 0 /Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f01/pipelineout/f_psm_01_d_01_out.fits
/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/ANALYSIS/PLOTS/ already exists. Skipping
530.952555194 532.686232574
Subtracted min value: -320.69493193
Saved file: /Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/ANALYSIS/PLOTS/f_psm_01_d_01_out.fits_f01.png
Saved file: /Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/ANALYSIS/PLOTS/f_psm_01_d_01_out.fits_f01.csv
Radii at EE: 314.0
Saved backup to /Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/ANALYSIS/CSV_RESULTS/radii_fwhm_subtracter.npz
FWHM 387.910637368

Analyzing file # 1 /Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f01/pipelineout/f_psm_01_d_02_out.fits
/Users/gks/Dropbox/mypylib/data/HPF_TES

,filename,x_out_fiber_dist,x_out_fiber_dist_delta,exptime,iris_diam_perc,iris,f_ratio_in,Unnamed: 7,radii_at_EE,sub_values,y_out_fiber_dist,y_out_fiber_dist_delta,f_ratio_out
0,f_psm_01_d_01.fits,32.0,0.0,1.0,18.0,8.024822,2.492267,2.0,314.0,-320.694932,4.082,0.000,NaN
1,f_psm_01_d_02.fits,31.0,1.0,1.0,18.0,8.024822,2.492267,2.0,329.0,-282.891856,4.277,0.195,2.564103
2,f_psm_01_d_03.fits,30.0,2.0,1.0,18.0,8.024822,2.492267,2.0,343.0,-273.377913,4.459,0.377,2.652520
3,f_psm_01_d_04.fits,29.0,3.0,1.0,18.0,8.024822,2.492267,2.0,358.0,-250.656463,4.654,0.572,2.622378
4,f_psm_02_d_01.fits,32.0,0.0,1.0,16.0,7.358418,2.717975,2.0,294.0,-298.691581,3.822,0.000,NaN
5,f_psm_02_d_02.fits,31.0,1.0,1.0,16.0,7.358418,2.717975,2.0,308.0,-278.133329,4.004,0.182,2.747253
6,f_psm_02_d_03.fits,30.0,2.0,1.0,16.0,7.358418,2.717975,2.0,322.0,-309.988686,4.186,0.364,2.747253
7,f_psm_02_d_04.fits,29.0,3.0,1.0,16.0,7.358418,2.717975,2.0,335.0,-255.023028,4.355,0.533,2.814259
8,f_psm_03_d_01.fits,32.0,0.0,1.0,14.0,6.692015,2.988637,2.0,274.0,-244.192043,3.562,0.000,NaN
9,f_psm_03_d_02.fits,31.0,1.0,1.0,14.0,6.692015,2.988637,2.0,287.0,-262.419214,3.731,0.169,2.958580


# F2

In [29]:
AFRDImg = AnalyzeFRDImages(fitsfiles=fitsfiles_f02,
                           plot_suffix="f02",
                           plot_folder=PLOT_FOLDER,
                           setup_csv_file=setup_csv_files[1],
                           FOLDER_CSV_SAVE=FOLDER_CSV_SAVE,
                           motorized=True,
                           MAXRAD_FACTOR=0.54)
AFRDImg.analyze_all_frames()

Analyzing file # 0 /Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f02/pipelineout/f_psm_01_d_01_out.fits
/Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/ANALYSIS/PLOTS/ already exists. Skipping
497.396574511 519.185344917
Subtracted min value: -290.247688006
Saved file: /Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/ANALYSIS/PLOTS/f_psm_01_d_01_out.fits_f02.png
Saved file: /Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/ANALYSIS/PLOTS/f_psm_01_d_01_out.fits_f02.csv
Radii at EE: 312.0
Saved backup to /Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/ANALYSIS/CSV_RESULTS/f02_radii_fwhm_subtracted.npz
FWHM 371.849246729

Analyzing file # 1 /Users/gks/Dropbox/mypylib/data/HPF_TEST_FIBER_FRD/20170803_test_puck_C_200micron_hene/f02/pipelineout/f_psm_01_d_02_out.fits
/Users/gks/Dropbox/mypylib/data/HP

,filename,x_out_fiber_dist,x_out_fiber_dist_delta,exptime,iris_diam_perc,iris,f_ratio_in,Unnamed: 7,radii_at_EE,sub_values,y_out_fiber_dist,y_out_fiber_dist_delta,f_ratio_out
0,f_psm_01_d_01.fits,32.0,0.0,1.0,18.0,8.024822,2.492267,2.0,312.0,-290.247688,4.056,0.000,NaN
1,f_psm_01_d_02.fits,31.0,1.0,1.0,18.0,8.024822,2.492267,2.0,326.0,-301.195089,4.238,0.182,2.747253
2,f_psm_01_d_03.fits,30.0,2.0,1.0,18.0,8.024822,2.492267,2.0,342.0,-367.027843,4.446,0.390,2.564103
3,f_psm_01_d_04.fits,29.0,3.0,1.0,18.0,8.024822,2.492267,2.0,356.0,-301.685113,4.628,0.572,2.622378
4,f_psm_02_d_01.fits,32.0,0.0,1.0,16.0,7.358418,2.717975,2.0,292.0,-290.289276,3.796,0.000,NaN
5,f_psm_02_d_02.fits,31.0,1.0,1.0,16.0,7.358418,2.717975,2.0,306.0,-281.586575,3.978,0.182,2.747253
6,f_psm_02_d_03.fits,30.0,2.0,1.0,16.0,7.358418,2.717975,2.0,320.0,-289.004500,4.160,0.364,2.747253
7,f_psm_02_d_04.fits,29.0,3.0,1.0,16.0,7.358418,2.717975,2.0,334.0,-277.939504,4.342,0.546,2.747253
8,f_psm_03_d_01.fits,32.0,0.0,1.0,14.0,6.692015,2.988637,2.0,273.0,-269.617312,3.549,0.000,NaN
9,f_psm_03_d_02.fits,31.0,1.0,1.0,14.0,6.692015,2.988637,2.0,286.0,-264.239367,3.718,0.169,2.958580
